In [17]:
from collections import defaultdict
import time
import random
import torch
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("../data/classes/train.txt"))


w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [19]:
print(len(xtrain[0]), len(ytrain))

36 8544


In [27]:
from torch.autograd import Variable
ftype = torch.cuda.FloatTensor
ltype = torch.cuda.LongTensor

num_epochs = 100

In [ ]:
import torch.nn as nn

EMB_SIZE = 64
HID_SIZE = 64
num_epochs = 100

model = torch.nn.Sequential(
    nn.EmbeddingBag(nwords, EMB_SIZE, mode='sum'),
    nn.Linear(EMB_SIZE, HID_SIZE),
    nn.Tanh(),
    nn.Linear(HID_SIZE, HID_SIZE),
    nn.Tanh(),
    nn.Linear(HID_SIZE, ntags)
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    
    #Perform training
    train_loss = 0.0
    random.shuffle(train)
    start = time.time()
    
    for words, tag in train:
        x = Variable(torch.LongTensor([words]), requires_grad = False)
        y = Variable(torch.LongTensor([tag]), requires_grad = False)
        
        # forward pass
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        train_loss += loss.data[0]
        
        # backward pass
        model.zero_grad()
        loss.backward()
        optimizer.step()

        #trainer.update()
        
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (epoch, train_loss/len(train), time.time()-start))
    
    # Perform testing
    test_correct = 0.0
    
    for words, tag in dev:
        x = Variable(torch.LongTensor([words]), requires_grad = False)
        y = Variable(torch.LongTensor([tag]), requires_grad = False)
        y_pred = model(x)
        if y_pred == y:
            test_correct += 1
    
    print("iter %r: test acc=%.4f" % (epoch, test_correct/len(dev)))